In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import xgboost as xgb

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score

In [3]:
tweets = pd.read_json('../input/wine-preferences/tweets_with_features.json')

In [5]:
tweets.head()

,username,description,date,location,following,followers,totaltweets,retweet,retweetcount,text,hashtags,brand,color,country,location_country,location_city
0,1267886436080107521,Get your store up: https://t.co/qf78jAU5eo,2022-10-03 18:06:57,United States,15,41,6223,False,0,repurposedupcycled green wine bottle base glas...,"[Repurposedupcycled, Glasses, set, Bottle, win...",None,None,None,[United States],[]
1,406344750,"The Best Tasting Wine Shop, a retail store wit...",2022-10-03 18:05:51,302 Pier Avenue,1939,995,2839,False,0,thanks much linda uncorked uncorkedwineshops w...,"[Uncorked, UncorkedWineShops, wineshop, shoplo...",None,None,None,[],[Avenue]
2,83460300,Love wine... Love champagne... Love life...,2022-10-03 18:05:48,"Sydney, Australia",1107,589,3456,False,0,louis jadot charmes chambertin grand cru gtgtg...,"[AD, wine, redwine, whitewine, whisky]",None,[red],None,[Australia],[Sydney]
3,853779552329502720,We tweet and retweet all things cool and inter...,2022-10-03 18:05:07,,362,191,61865,False,7,deals events make feel larger life httpstcoscc...,"[soulnightevents, party]",None,None,None,None,None
4,838972202,"Serial entrepreneur, that's taken two successf...",2022-10-03 18:05:05,"Delray Beach, FL",1029,8772,16697,False,0,become wine influencer six figures per year wi...,"[wine, winelovers, winetasting, wineisfun, fun...",None,None,None,[],"[Delray Beach, Beach]"


In [6]:
xgb_model = xgb.Booster()
xgb_model.load_model('../input/wine-preferences/big_model.json')

In [7]:
import pickle

with open('../input/wine-preferences/vmodel.txt', 'rb') as f:
    cv = pickle.load(f)

#cv_train = pd.read_csv('../input/wine-preferences/train_cleaned.csv', converters={"text": lambda x: ' '.join(x.strip("[]").replace("'", '').split(", "))}, index_col=0)

#cv = CountVectorizer(binary = True)
#cv.fit(cv_train['text'])

In [8]:
tweets_x = cv.transform(tweets['text'])
xgb_tweets = xgb.DMatrix(tweets_x)

In [9]:
tweets_y = xgb_model.predict(xgb_tweets)
tweets['sentiment'] = np.where(np.array(tweets_y) > 0.5, 1, 0)

In [10]:
tweets.head()

,username,description,date,location,following,followers,totaltweets,retweet,retweetcount,text,hashtags,brand,color,country,location_country,location_city,sentiment
0,1267886436080107521,Get your store up: https://t.co/qf78jAU5eo,2022-10-03 18:06:57,United States,15,41,6223,False,0,repurposedupcycled green wine bottle base glas...,"[Repurposedupcycled, Glasses, set, Bottle, win...",None,None,None,[United States],[],0
1,406344750,"The Best Tasting Wine Shop, a retail store wit...",2022-10-03 18:05:51,302 Pier Avenue,1939,995,2839,False,0,thanks much linda uncorked uncorkedwineshops w...,"[Uncorked, UncorkedWineShops, wineshop, shoplo...",None,None,None,[],[Avenue],0
2,83460300,Love wine... Love champagne... Love life...,2022-10-03 18:05:48,"Sydney, Australia",1107,589,3456,False,0,louis jadot charmes chambertin grand cru gtgtg...,"[AD, wine, redwine, whitewine, whisky]",None,[red],None,[Australia],[Sydney],1
3,853779552329502720,We tweet and retweet all things cool and inter...,2022-10-03 18:05:07,,362,191,61865,False,7,deals events make feel larger life httpstcoscc...,"[soulnightevents, party]",None,None,None,None,None,1
4,838972202,"Serial entrepreneur, that's taken two successf...",2022-10-03 18:05:05,"Delray Beach, FL",1029,8772,16697,False,0,become wine influencer six figures per year wi...,"[wine, winelovers, winetasting, wineisfun, fun...",None,None,None,[],"[Delray Beach, Beach]",1


In [11]:
tweets.to_csv('tweets_sentiment.csv')